# <center> **Classificação de sons de motores**

In [ ]:
#https://www.kaggle.com/sachinsarkar/urban-sound-classification-usnig-librosa-and-ann/notebook
import os
import numpy as np
import pandas as pd
import librosa as lb
import IPython.display as ipd
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sns
from tensorflow import keras
from tensorflow.keras import Sequential, layers
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from tqdm import tqdm

In [ ]:
#Carregar um exemplo de arquivo de áudio e mplotar a onda
def plot_sound(filename):
    librosa_audio_data, librosa_sample_rate = lb.load(filename)
    time_x = np.arange(len(librosa_audio_data)) / librosa_sample_rate
    plt.plot(time_x, librosa_audio_data)
    plt.xlabel("Tempo (s)")

filename = './motoserras/07002325.wav'
plot_sound(filename)

ipd.Audio(filename)

#### A function that extract and returns numeric features from audio file

In [ ]:
# Corta um arquivo de som em vetores menores de t segundos
def split_data(path):
    # IMPORTANTE: não foi verificado o conteúdo dos áudios de 5 segundos, provavelmente
    # há trechos de sons que devem ser removidos (exemplo: ruídos)
    t = 20
    data, sample_rate = lb.load(path)

    audio_slice_list = []
    for i in range(0, len(data), t * sample_rate):
        audio_slice_list.append(data[i : t * sample_rate + i])
    return audio_slice_list

# Extrai features do arquivo de som
def feature_extractor(data):
    # data, sample_rate = lb.load(path)

    data = lb.feature.mfcc(y=data, n_mfcc=128)
    data = np.mean(data,axis=1)
    return data

#### Extracting Features from Audio files and preparing the dataset

In [ ]:
# Loading sounds from 2 different folders
path_motosserras = "./motoserras"
path_motores = "./motores"

In [ ]:
x, y = [], []

# Carrega todos os sons de motores
for filename in os.listdir(path_motores):
    path = os.path.join(path_motores, filename)
    audio_slice_list = split_data(path)
    for audio in audio_slice_list:
        x.append(feature_extractor(audio))
        y.append(0)

# Carrega todos os sons de motoserras
for filename in os.listdir(path_motosserras):
    path = os.path.join(path_motosserras, filename)
    audio_slice_list = split_data(path) 
    for audio in audio_slice_list:
        x.append(feature_extractor(audio))
        y.append(1)

x = np.array(x)
y = np.array(y)
x.shape, y.shape



#### Train, Test and validation Split

In [ ]:
xtrainval, xtest, ytrainval, ytest = train_test_split(x,y,test_size=0.1,stratify=None,random_state=0)
xtrain, xvalid, ytrain, yvalid = train_test_split(xtrainval,ytrainval,test_size=0.2, random_state=1)

print('\nNumber of samples for Train set :',xtrain.shape[0])
print('Number of samples for Validation set :',xvalid.shape[0])
print('Number of samples for Test set :',xtest.shape[0])


#### Artificial Neural Network Model Building

In [ ]:
model = Sequential(
                        [
                            layers.Dense(1000,activation='relu',input_shape=(128,)),
                            layers.Dense(750,activation='relu'),
                            layers.Dense(500,activation='relu'),
                            layers.Dense(250,activation='relu'),
                            layers.Dense(100,activation='relu'),
                            layers.Dense(50,activation='relu'),
                            layers.Dense(1,activation='softmax')
                        ]
                   )
model.summary()

#### Training and Compilation of the model

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
training = model.fit(xtrain,ytrain,validation_data=(xvalid,yvalid),epochs=20)

#### Training History

In [ ]:
train_hist = pd.DataFrame(training.history)
train_hist

#### Visualizing Training History

In [ ]:
plt.figure(figsize=(20,8))
plt.plot(train_hist[['loss','val_loss']])
plt.legend(['loss','val_loss'])
plt.title('Loss Per Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.show()

plt.figure(figsize=(20,8))
plt.plot(train_hist[['accuracy','val_accuracy']])
plt.legend(['accuracy','val_accuracy'])
plt.title('Accuracy Per Epochs')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.show()

#### Model Performance Analysis on Test Data

In [ ]:
ytrue = ytest
ypred = model.predict(xtest)[:, 0]
ypred = np.round(ypred)

print('\n\nClassification Report : \n\n',classification_report(ytrue,ypred))

In [ ]:
plt.figure(figsize=(10,4))
plt.title("Confusion matrix for testing data", fontsize = 15)
plt.xlabel("Predicted class")
plt.ylabel("True class")
sns.heatmap(confusion_matrix(ytrue,ypred),annot=True,
           xticklabels = ['motor', 'motoserra'], yticklabels=['motor', 'motoserra'])

plt.show()

#### The final Prediction function that takes the audio path and returns the predicted class along with audio

In [ ]:
def predict(path):
    # Lê um arquivo de áudio e separa ele em partes
    audio_slice_list = split_data(path)
    audio = np.array([feature_extractor(audio_slice_list[0])])
    classid = model.predict(audio)[0][0]
    classe = 'motosserra' if classid == 1 else 'motor'
    print('Classe predita :',classe,'\n\n')
    return ipd.Audio(path)

#### Testing the Prediction Function on a Audio file

In [ ]:
predict('./motoserras/07002291.wav')

# <center> **Thank You**